In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd

# Multi-dimensional arrays and datasets
import xarray as xr

# Geospatial raster data handling
import rioxarray as rxr

# Geospatial data analysis
import geopandas as gpd

# Geospatial operations
import rasterio
from rasterio import windows  
from rasterio import features  
from rasterio import warp
from rasterio.warp import transform_bounds 
from rasterio.windows import from_bounds 

import pyogrio
from shapely.geometry import Point

# Image Processing
from PIL import Image

# Coordinate transformations
from pyproj import Proj, Transformer, CRS

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer

# Feature Importance
import shap 

# Machine Learning
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from xgboost import XGBRegressor 
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor

import torch
from pytorch_tabnet.tab_model import TabNetRegressor    
from pytorch_tabnet.metrics import Metric

from tabpfn import TabPFNRegressor

# from sklearn.linear_model import Ridge, Lasso, ElasticNet

# Hyperparameter Tuning
import optuna

# Planetary Computer Tools
import pystac_client
import planetary_computer as pc
from pystac.extensions.eo import EOExtension as eo

# Others
import os
from tqdm import tqdm

In [ ]:
import pandas as pd

df = pd.read_csv('UHI_data.csv')
bronx_df = pd.read_excel('NY_Mesonet_Weather.xlsx', sheet_name='Bronx')
manhattan_df = pd.read_excel('NY_Mesonet_Weather.xlsx', sheet_name='Manhattan')
submission = pd.read_csv('Validation_with_coords.csv')

In [ ]:
def merge_train_data(df, bronx, manhattan, tolerance=0.01, bronx_lat=40.87248,
               bronx_lon=-73.89352, manhattan_lat=40.76754, manhattan_lon=-73.96449):

    df['datetime'] = pd.to_datetime(df['datetime'], format='%d-%m-%Y %H:%M')

    bronx['Date / Time'] = pd.to_datetime(bronx['Date / Time'])
    manhattan['Date / Time'] = pd.to_datetime(manhattan['Date / Time'])

    bronx.rename(columns={'Date / Time': 'datetime'}, inplace=True)
    manhattan.rename(columns={'Date / Time': 'datetime'}, inplace=True)

    # Filter CSV for Manhattan based on coordinate tolerance:
    manhattan_csv = df[
        (df['Latitude'].between(manhattan_lat - tolerance, manhattan_lat + tolerance)) |
        (df['Longitude'].between(manhattan_lon - tolerance, manhattan_lon + tolerance))
    ]

    bronx_csv = df[
        (df['Latitude'].between(bronx_lat - tolerance, bronx_lat + tolerance)) |
        (df['Longitude'].between(bronx_lon - tolerance, bronx_lon + tolerance))
    ]

    print("Manhattan CSV rows found:", len(manhattan_csv))
    print("Bronx CSV rows found:", len(bronx_csv))

    # Merge the temperature data from Excel with the corresponding CSV data based on datetime.
    # For Manhattan:
    merged_manhattan = pd.merge(manhattan_csv, manhattan, on='datetime', how='inner')
    print("Merged Manhattan data:")
    print(merged_manhattan.head())

    # For Bronx, if any rows are found; if not, consider a wider tolerance
    if not bronx_csv.empty:
        merged_bronx = pd.merge(bronx_csv, bronx, on='datetime', how='inner')
        print("Merged Bronx data:")
        print(merged_bronx.head())
    else:
        print("No Bronx rows found in CSV with tolerance of ±0.01. Consider increasing tolerance.")

    manhattan_lookup = merged_manhattan[['Longitude', 'Latitude',
                                      'Air Temp at Surface [degC]',
                                      'Relative Humidity [percent]',
                                      'Avg Wind Speed [m/s]',
                                      'Wind Direction [degrees]',
                                      'Solar Flux [W/m^2]']]

    bronx_lookup = merged_bronx[['Longitude', 'Latitude',
                                'Air Temp at Surface [degC]',
                                'Relative Humidity [percent]',
                                'Avg Wind Speed [m/s]',
                                'Wind Direction [degrees]',
                                'Solar Flux [W/m^2]']]

    # Combine (concatenate) the two lookup DataFrames into one.
    lookup = pd.concat([manhattan_lookup, bronx_lookup])

    # Now merge the main DataFrame (uhi) with the combined lookup DataFrame on the key columns.
    df = pd.merge(df, lookup, on=['Longitude', 'Latitude'], how='left')

    return df


In [ ]:
uhi = merge_train_data(df, bronx_df, manhattan_df, tolerance=0.001)

In [ ]:
imputer = KNNImputer(n_neighbors=10, weights='distance')

In [ ]:
band_cols = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 
                'B07', 'B08', 'B8A', 'B11', 'B12']
weather_cols = [
    'Air Temp at Surface [degC]',
    'Relative Humidity [percent]',
    'Avg Wind Speed [m/s]',
    'Wind Direction [degrees]',
    'Solar Flux [W/m^2]'
]

coord_cols=['Longitude', 'Latitude']

impute_cols = coord_cols + band_cols + weather_cols

# Check that all required columns exist in uhi.
missing_cols = [col for col in impute_cols if col not in uhi.columns]
if missing_cols:
    raise ValueError(f"The following columns are missing from the DataFrame: {missing_cols}")

# Create a copy of the relevant data for imputation.
data_to_impute = uhi[impute_cols].copy()

# Initialize the KNNImputer.
imputer = KNNImputer(n_neighbors=10, weights='distance')

# Fit and transform the data.
imputed_array = imputer.fit_transform(data_to_impute)

# The imputed_array has the same order as impute_cols.
# Extract the imputed weather data columns. They are at the end of the array.
weather_start_idx = len(coord_cols) + len(band_cols)
imputed_weather = imputed_array[:, weather_start_idx:]

uhi[weather_cols] = imputed_weather

In [ ]:
for col in weather_cols:
    if col not in submission.columns:
        submission[col] = np.nan

test_for_impute = submission[impute_cols].copy()

# Use the trained imputer to transform the test data.
imputed_array = imputer.transform(test_for_impute)

# The imputer was trained on data with columns in the order:
#   coord_cols + band_cols + weather_cols.
# Extract the imputed weather columns (the last len(weather_cols) columns).
imputed_weather = imputed_array[:, -len(weather_cols):]

# Update the test DataFrame's weather columns with the imputed values.
submission[weather_cols] = imputed_weather
    

In [ ]:
# Split the data into features (X) and target (y), and then into training and testing sets
X = uhi.drop(columns=['Longitude','Latitude','datetime','UHI Index']).values
y = uhi['UHI Index'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [ ]:
# def objective_xg(trial):
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 100, 1500),
#         'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
#         'max_depth': trial.suggest_int('max_depth', 3, 20),
#         'subsample': trial.suggest_float('subsample', 0.5, 1),
#         'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
#         'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
#         # 'tree_method': trial.suggest_categorical('tree_method', ['exact', 'approx', 'hist'])
#     }
    
#     model = XGBRegressor(**params, random_state=42, tree_method='exact')
#     # model.fit(X_train, y_train)

#     # preds = model.predict(X_test)
#     # score = r2_score(y_test, preds)
#     cv = KFold(n_splits=10, shuffle=True, random_state=42)
#     scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2')
#     score = scores.mean()
    
#     return score

# study_xg = optuna.create_study(direction='maximize')  
# study_xg.optimize(objective_xg, n_trials=100)

# print("Best Hyperparameters:", study_xg.best_params)
# print('Best Score:', study_xg.best_value)

# best_params_xg = study_xg.best_params

# with open("xg_results.txt", "a") as f:
#     f.write(f"Best Score: {study_xg.best_value}\n")
#     f.write(f"Best Hyperparameters: {study_xg.best_params}\n")

In [ ]:
# def objective_lgbm(trial):
#     # params = {
#     #     'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
#     #     'max_depth': trial.suggest_int('max_depth', 3, 20),
#     #     'num_leaves': trial.suggest_int('num_leaves', 10, 150),
#     #     'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
#     #     'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
#     #     'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
#     #     'bagging_freq': trial.suggest_int('bagging_freq', 3, 7),
#     #     'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),  
#     #     'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
#     #     'n_estimators': trial.suggest_int('n_estimators', 100, 1000)  
#     # }
#     params = {
#         'learning_rate': trial.suggest_float('learning_rate', 1e-6, 0.5),      
#         'max_depth': trial.suggest_int('max_depth', 2, 50),                   
#         'num_leaves': trial.suggest_int('num_leaves', 5, 300),                 
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 200),       
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),   
#         'bagging_freq': trial.suggest_int('bagging_freq', 0, 10),               
#         'lambda_l1': trial.suggest_float('lambda_l1', 0, 50),                    
#         'lambda_l2': trial.suggest_float('lambda_l2', 0, 50),
#         'n_estimators': trial.suggest_int('n_estimators', 50, 1500)              
#     }


#     model = LGBMRegressor(**params, random_state=42, verbose=-1)
#     # model.fit(X_train, y_train)

#     # preds = model.predict(X_test)
#     # score = r2_score(y_test, preds)

#     cv = KFold(n_splits=10, shuffle=True, random_state=42)
#     scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2')
#     score = scores.mean()

#     return score

# study_lgbm = optuna.create_study(direction='maximize')
# study_lgbm.optimize(objective_lgbm, n_trials=1000)

# print('Best Hyperparameters:', study_lgbm.best_params)
# print('Best Score:', study_lgbm.best_value)

# best_params_lgbm = study_lgbm.best_params

# with open("lgbm_results.txt", "a") as f:
#     f.write(f"Best Score: {study_lgbm.best_value}\n")
#     f.write(f"Best Hyperparameters: {study_lgbm.best_params}\n")

In [ ]:
# def objective_rf(trial):
#     params = {
#         'n_estimators' : trial.suggest_int("n_estimators", 100, 2000),
#         'max_depth' : trial.suggest_int("max_depth", 5, 30),
#         'min_samples_split' : trial.suggest_int("min_samples_split", 2, 20),
#         'min_samples_leaf' : trial.suggest_int("min_samples_leaf", 2, 20),
#         'max_features' : trial.suggest_categorical("max_features", ["sqrt", "log2"])
#     }
    
#     model = RandomForestRegressor(**params, random_state=42, n_jobs=-1)
#     # model.fit(X_train, y_train)

#     # preds = model.predict(X_test)
#     # score = r2_score(y_test, preds)
#     cv = KFold(n_splits=10, shuffle=True, random_state=42)
#     scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2')
#     score = scores.mean()
    
#     return score

# study_rf = optuna.create_study(direction='maximize')  
# study_rf.optimize(objective_rf, n_trials=100)

# print("Best Hyperparameters:", study_rf.best_params)
# print('Best Score:', study_rf.best_value)

# best_params_rf = study_rf.best_params

# with open("rf_results.txt", "a") as f:
#     f.write(f"Best Score: {study_rf.best_value}\n")
#     f.write(f"Best Hyperparameters: {study_rf.best_params}\n")

In [ ]:
# def objective_cat(trial):
#     params = {
#         "iterations": trial.suggest_int("iterations", 500, 2000, step=100),
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
#         "depth": trial.suggest_int("depth", 4, 12),
#         "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-5, 10, log=True),
#         "random_strength": trial.suggest_float("random_strength", 0, 10),
#         "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 1),
#         "border_count": trial.suggest_int("border_count", 32, 255),
#         "loss_function": "RMSE",
#         "eval_metric": "RMSE",
#         "random_seed": 42,
#         "verbose": 0
#     }
    
#     model = CatBoostRegressor(**params)
#     # model.fit(X_train, y_train)

#     # preds = model.predict(X_test)
#     # score = r2_score(y_test, preds)
#     cv = KFold(n_splits=10, shuffle=True, random_state=42)
#     scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2')
#     score = scores.mean()
    
#     return score

# study_cat = optuna.create_study(direction='maximize')  
# study_cat.optimize(objective_cat, n_trials=100)

# print("Best Hyperparameters:", study_cat.best_params)
# print('Best Score:', study_cat.best_value)

# best_params_cat = study_cat.best_params

# with open("cat_results.txt", "a") as f:
#     f.write(f"Best Score: {study_cat.best_value}\n")
#     f.write(f"Best Hyperparameters: {study_cat.best_params}\n")

In [ ]:
# def objective_gbr(trial):
#     params = {
#         #'loss':trial.suggest_categorical('loss', ['squared_error', 'huber', 'quantile', 'absolute_error']),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1.0),
#         'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
#         'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
#         #'criterion': trial.suggest_categorical('criterion', ['friedman_mse', 'squared_error']),
#         'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
#         'max_depth': trial.suggest_int('max_depth', 3, 20),
#     }
    
#     model = GradientBoostingRegressor(**params, random_state=42, criterion='squared_error')
#     # model.fit(X_train, y_train)

#     # preds = model.predict(X_test)
#     # score = r2_score(y_test, preds)
#     cv = KFold(n_splits=10, shuffle=True, random_state=42)
#     scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2')
#     score = scores.mean()
    
#     return score

# study_gbr = optuna.create_study(direction='maximize')  
# study_gbr.optimize(objective_gbr, n_trials=100)

# print("Best Hyperparameters:", study_gbr.best_params)
# print('Best Score:', study_gbr.best_value)

# best_params_gbr = study_gbr.best_params

# with open("gbr_results.txt", "a") as f:
#     f.write(f"Best Score: {study_gbr.best_value}\n")
#     f.write(f"Best Hyperparameters: {study_gbr.best_params}\n")


In [ ]:
# def objective_hgbr(trial):
#     params = {
#         'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1.0),
#         'max_iter': trial.suggest_int('max_iter', 100, 2000),
#         'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 200),   
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 100),
#         'max_depth': trial.suggest_int('max_depth', 3, 20),
#         'l2_regularization': trial.suggest_loguniform('l2_regularization', 1e-5, 1.0),
#         'max_bins': trial.suggest_int('max_bins', 100, 255),
#     }
    
#     model = HistGradientBoostingRegressor(**params, random_state=42, loss='squared_error')
#     # model.fit(X_train, y_train)

#     # preds = model.predict(X_test)
#     # score = r2_score(y_test, preds)
#     cv = KFold(n_splits=10, shuffle=True, random_state=42)
#     scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2')
#     score = scores.mean()
    
#     return score

# study_hgbr = optuna.create_study(direction='maximize')  
# study_hgbr.optimize(objective_hgbr, n_trials=100)

# print("Best Hyperparameters:", study_hgbr.best_params)
# print('Best Score:', study_hgbr.best_value)

# best_params_hgbr = study_hgbr.best_params

# with open("hgbr_results.txt", "a") as f:
#     f.write(f"Best Score: {study_hgbr.best_value}\n")
#     f.write(f"Best Hyperparameters: {study_hgbr.best_params}\n")


In [ ]:
best_params_xg = {'n_estimators': 527, 'learning_rate': 0.0312788331848764, 'max_depth': 17, 'subsample': 0.748120538929842, 'reg_alpha': 0.001241366749156092, 'reg_lambda': 4.027151002258514}
best_params_lgbm = {'learning_rate': 0.049306462164681195, 'max_depth': 12, 'num_leaves': 145, 'min_data_in_leaf': 8, 'feature_fraction': 0.8758609827849215, 'bagging_fraction': 0.24707848190077422, 'bagging_freq': 0, 'lambda_l1': 0.0029913663437004133, 'lambda_l2': 6.191482291982284, 'n_estimators': 1318}
best_params_rf = {'n_estimators': 713, 'max_depth': 29, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'log2'}

In [ ]:
# model = XGBRegressor(**best_params_xg, random_state=42, tree_method='exact')
# model = LGBMRegressor(**best_params_lgbm, random_state=42, verbose=-1)
# model = CatBoostRegressor(**best_params_cat, verbose=False)
# model = GradientBoostingRegressor(**best_params_gbr, random_state=42, criterion='squared_error')
# model = HistGradientBoostingRegressor(**best_params_hgbr, random_state=42, loss='squared_error')
# model = RandomForestRegressor(**best_params_rf, random_state=42, n_jobs=-1)
# estimators = [
#     ('xgb', XGBRegressor(**best_params_xg, random_state=42, tree_method='exact')),
    # ('cat', CatBoostRegressor(**best_params_cat, verbose=False)),
    # ('lgb', LGBMRegressor(**best_params_lgbm, random_state=42, verbose=-1)),
    # ('gbr', GradientBoostingRegressor(**best_params_gbr, random_state=42, criterion='squared_error')),
    # ('hgbr', HistGradientBoostingRegressor(**best_params_hgbr, random_state=42, loss='squared_error'))
# ]
# model = StackingRegressor(
#     estimators=estimators, 
#     final_estimator=RandomForestRegressor(**best_params_rf, random_state=42, n_jobs=-1),
    # final_estimator=XGBRegressor(**best_params_xg, random_state=42, tree_method='exact'),
    # final_estimator=GradientBoostingRegressor(**best_params_gbr, random_state=42, criterion='squared_error'),
#     cv=10,
#     passthrough=True
# )

In [ ]:
model.fit(X_train, y_train)

In [ ]:
insample_predictions = model.predict(X_train)
Y_train = y_train.tolist()
r2_score(Y_train, insample_predictions)

In [ ]:
outsample_predictions = model.predict(X_test)
Y_test = y_test.tolist()
r2_score(Y_test, outsample_predictions)

In [ ]:
feature_columns = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 
                   'B07', 'B08', 'B8A', 'B11', 'B12', 'LST', 'is_building', 
                   'Air Temp at Surface [degC]', 'Relative Humidity [percent]', 
                   'Avg Wind Speed [m/s]', 'Wind Direction [degrees]', 'Solar Flux [W/m^2]']

In [ ]:
# For example, if submission originally has more columns, select only the ones used in training:
submission_prepared = submission[feature_columns].copy()

# Now predict:
final_predictions = model.predict(submission_prepared)

final_prediction_series = pd.Series(final_predictions)

In [ ]:
sub = pd.read_csv('Submission_template.csv')

In [ ]:
submission_df = pd.DataFrame({'Longitude':sub['Longitude'].values, 'Latitude':sub['Latitude'].values, 'UHI Index':final_prediction_series.values})

In [ ]:
submission_df.to_csv('submission.csv', index=False)